In [1]:
%reload_ext autoreload
%autoreload 2

## Experiment EXP-23-CA3091, EXP-24-CB4713, EXP-24-CB4719

In [2]:
import json
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from pathlib import Path
%matplotlib qt5

In [3]:
all_data = []

In [4]:
### Number of beads:
DATA_DIR = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-23-CA3091/beadsLocalizationAccuracy/lowIntensityLocalizations/')
for i in range(1, 6):
    with open(DATA_DIR / Path('Pos' + str(i)), 'r') as fh:
        data = json.load(fh)
        print(f"Number of beads: {len(data)}")
        if len(data) != 0:
            all_data.extend(data)

Number of beads: 4
Number of beads: 9
Number of beads: 16
Number of beads: 29
Number of beads: 14


In [5]:
DATA_DIR = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4713/fluorChannelRegistration3D/localizations_vs_z/')
for filename in sorted(list(DATA_DIR.glob('*'))):
    with open(filename, 'r') as fh:
        data = json.load(fh)
    #print(f"Number of beads: {len(data)}")
        if len(data) != 0:
            all_data.extend(data)

In [6]:
DATA_DIR = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4719/fluorChannelRegistration3D/localizations_vs_z_101/')
for filename in sorted(list(DATA_DIR.glob('*'))):
    with open(filename, 'r') as fh:
        data = json.load(fh)
        #print(f"Number of beads: {len(data)}")
        if len(data) != 0:
            all_data.extend(data)

In [7]:
DATA_DIR = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4719/fluorChannelRegistration3D/localizations_vs_z_102/')
for filename in sorted(list(DATA_DIR.glob('*'))):
    with open(filename, 'r') as fh:
        data = json.load(fh)
        #print(f"Number of beads: {len(data)}")
        if len(data) != 0:
            all_data.extend(data)

In [8]:
DATA_DIR = Path('/mnt/sda1/SMLAT/data/real_data/only_beads/EXP-24-CB4719/fluorChannelRegistration3D/localizations_vs_z_103/')
for filename in sorted(list(DATA_DIR.glob('*'))):
    with open(filename, 'r') as fh:
        data = json.load(fh)
        #print(f"Number of beads: {len(data)}")
        if len(data) != 0:
            all_data.extend(data)

In [9]:
len(all_data)

1701

In [10]:
all_data[0].keys()

dict_keys(['bead_no', 'x', 'y', 'z', 'ph', 'x_sigma', 'y_sigma', 'z_sigma', 'ph_sigma'])

In [11]:
all_data[0]['ph']

[1653.92578125,
 1516.7547607421875,
 1555.9029541015625,
 1386.6798095703125,
 1339.0155029296875,
 1551.360595703125,
 1553.19189453125,
 1485.6495361328125,
 1760.02978515625,
 1440.072509765625,
 1221.911865234375,
 1626.4915771484375,
 1638.2347412109375,
 1447.280517578125,
 1722.836181640625,
 1609.3990478515625,
 1469.8367919921875,
 1595.677734375,
 1614.62646484375,
 1387.81884765625,
 1732.0,
 1853.8580322265625,
 1549.402587890625,
 1507.365966796875,
 1559.8046875,
 1691.17529296875,
 1640.1982421875,
 1397.6876220703125,
 1469.1204833984375,
 1279.19384765625,
 1455.131591796875,
 1600.40234375,
 1522.29833984375,
 1469.0806884765625,
 1551.463623046875,
 1634.6685791015625,
 1330.9814453125,
 1564.80078125,
 1502.6754150390625,
 1547.3507080078125,
 1534.42138671875,
 1479.6112060546875,
 1453.30322265625,
 1283.376953125,
 1169.40576171875,
 1371.272216796875,
 1160.95751953125,
 1192.0916748046875,
 1121.2764892578125,
 1308.333984375,
 1036.1951904296875,
 1150.366699

In [12]:
photon_counts = [np.mean(one_bead['ph']) for one_bead in all_data]

In [13]:
np.mean(photon_counts)

1820.8298292361446

In [14]:
np.min(photon_counts)

711.1393788655599

In [15]:
tuples = [(np.mean(one_bead['z']), np.std(one_bead['x']), np.std(one_bead['y']), np.std(one_bead['z']), len(one_bead['z']), np.mean(one_bead['ph']))
          for one_bead in all_data
                     if len(one_bead['z']) > 30 and np.mean(one_bead['ph']) > 700.0
         ]

In [16]:
counts = [len(one_bead['z']) for one_bead in all_data]


In [17]:
z_values = [bead[0] for bead in tuples]
x_std = [bead[1] for bead in tuples]
y_std = [bead[2] for bead in tuples]
z_std = [bead[3] for bead in tuples]
ph_counts = [bead[5] for bead in tuples]

In [18]:
np.mean(ph_counts)

1947.8840252890523

In [19]:
len(z_values), np.mean(ph_counts)

(1311, 1947.8840252890523)

plt.figure()
plt.plot(z_values, x_std, 'r*')
plt.show()

plt.figure()
plt.plot(z_values, y_std, 'b*')
plt.show()

plt.figure()
plt.plot(z_values, z_std, 'g*')
plt.show()

In [20]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import time
import sys
import pathlib
import pickle
from pathlib import Path
import matplotlib.animation as animation
import seaborn as sns
sns.set_style('white')
sys.setrecursionlimit(10000)
%matplotlib qt5

In [21]:
import cellbgnet
import cellbgnet.utils
from cellbgnet.datasets import DataSimulator
from cellbgnet.utils.hardware import cpu, gpu
from cellbgnet.model import CellBGModel
from cellbgnet.utils.plot_funcs import plot_od, plot_train_record
from cellbgnet.simulation.psf_kernel import SMAPSplineCoefficient
from cellbgnet.analyze_eval import spline_crlb_plot
from cellbgnet.utils.plot_funcs import plot_psf

In [22]:
model_path = Path('/mnt/sda1/SMLAT/training_runs/model_rotated_45_venus_beads2.pkl')
with open(model_path, 'rb') as f:
    chromo_model = pickle.load(f)

In [23]:
chromo_model.simulation_params

{'train_size': 128,
 'train_type': 'uniform',
 'psf_intensity_normalization': True,
 'density': 0.003125,
 'non_cell_density': 10,
 'margin_empty': 0.05,
 'z_prior': [-1, 1],
 'survival_prob': None,
 'min_photon': 0.15,
 'camera': 'sCMOS',
 'camera_chip_size': [1041, 1302],
 'perlin_noise': False,
 'perlin_noise_factor': 0.2,
 'perlin_noise_res': 64,
 'baseline': 103.0,
 'bg_values': 112.0,
 'bg_width': 20,
 'e_per_adu': 0.38,
 'qe': 0.95,
 'spurious_c': 0.0,
 'sig_read': '/mnt/sda1/SMLAT/data/real_data/dark_images/RN_e_venus_upgraded.tiff',
 'robust_training': False,
 'cell_masks_dir': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-23-CA3045/pool_3045_ter_pos01/phase_venus_mask/',
 'cell_masks_filetype': '.png',
 'edt_noise_map_path': '/mnt/sda1/SMLAT/data/real_data/chromosome_dots/EXP-23-CA3045/edt_noise.pkl',
 'min_edt': 0,
 'max_edt': 6,
 'augment': True}

In [24]:
mol_photons = (chromo_model.data_generator.simulation_params['min_photon'] + 1) /2 * chromo_model.data_generator.psf_params['photon_scale']

In [25]:
mol_photons

2875.0

In [62]:
calib_file = chromo_model.psf_params['calib_file']
crlb, rois = spline_crlb_plot(calib_file, z_range=600.0, photon_counts=1947.0, bg_photons=5.8, img_size=41)

Mean CRLB x over full z range:-600.0 - 600.0 [nm] is: 12.362032890319824
Mean CRLB y over full z range:-600.0 - 600.0 [nm] is  11.97386360168457
Mean CRLB z over full z range:-600.0 - 600.0 [nm] is  36.54850769042969


In [63]:
z = torch.arange(-600.0, 610.0, 10.0)

In [64]:
crlb.shape

torch.Size([121, 5])

In [65]:
z.shape

torch.Size([121])

In [66]:
len(x_std)

1311

In [67]:
fig, ax = plt.subplots(nrows=1, ncols = 3, figsize=(16,4))

ax[0].plot(z_values, x_std, 'b*', label = 'Bead data')
ax[0].plot(z, crlb[:, 0], 'r--', label = r'$\sqrt{CRLB_{x}}$ (nm) Spline model')
ax[0].set_xlim([-500, 500])
ax[0].set_ylim([0, 120])
ax[0].set_xlabel('Z (nm)', fontsize=16)
ax[0].set_ylabel('Localization precision in X (nm)', fontsize=16)
ax[0].legend(loc='upper right', fontsize=12)
ax[0].tick_params(labelsize=16)

ax[1].plot(z_values, y_std, 'g*', label = 'Bead data')
ax[1].plot(z, crlb[:, 1], 'r--', label = r'$\sqrt{CRLB_{y}}$ (nm) Spline model')
ax[1].set_xlim([-500, 500])
ax[1].set_ylim([0, 120])
ax[1].set_xlabel('Z (nm)', fontsize=16)
ax[1].set_ylabel('Localization precision in Y (nm)', fontsize=16)
ax[1].legend(loc='upper right', fontsize=12)
ax[1].tick_params(labelsize=16)

ax[2].plot(z_values, z_std, 'k*', label = 'Bead data')
ax[2].plot(z, crlb[:, 2], 'r--', label = r'$\sqrt{CRLB_{z}}$ (nm) Spline model')
ax[2].set_xlim([-500, 500])
ax[2].set_xlabel('Z (nm)', fontsize=16)
ax[2].set_ylabel('Localization precision in Z (nm)', fontsize=16)
ax[2].legend(loc='upper right', fontsize=12)
ax[2].tick_params(labelsize=16)

#fig.delaxes(ax[1, 1])
#plt.suptitle(f'N = {len(x_std)} bead total (each localized >= 30 times)', fontsize=16, verticalalignment='baseline')
plt.tight_layout()
plt.show()

In [68]:
bins = np.linspace(-500, 500, 21)

In [69]:
bins

array([-500., -450., -400., -350., -300., -250., -200., -150., -100.,
        -50.,    0.,   50.,  100.,  150.,  200.,  250.,  300.,  350.,
        400.,  450.,  500.])

In [70]:
digitized = np.digitize(z_values, bins)

In [71]:
digitized.shape

(1311,)

In [72]:
x_std = np.array(x_std)
y_std = np.array(y_std)
z_std = np.array(z_std)

In [73]:
from scipy.stats import median_abs_deviation

In [74]:
bin_means_x = np.array([(x_std[digitized == i].mean(), (bins[i-1] + bins[i])/2) for i in range(1, len(bins))])
bin_std_x = np.array([(median_abs_deviation(x_std[digitized == i]), (bins[i-1] + bins[i])/2) for i in range(1, len(bins))])
bin_means_y = np.array([(y_std[digitized == i].mean(), (bins[i-1] + bins[i])/2) for i in range(1, len(bins))])
bin_std_y = np.array([(median_abs_deviation(y_std[digitized == i]), (bins[i-1] + bins[i])/2) for i in range(1, len(bins))])
bin_means_z = np.array([(z_std[digitized == i].mean(), (bins[i-1] + bins[i])/2) for i in range(1, len(bins))])
bin_std_z = np.array([(median_abs_deviation(z_std[digitized == i]), (bins[i-1] + bins[i])/2) for i in range(1, len(bins))])


In [75]:
bin_std_x

array([[   4.8218895 , -475.        ],
       [   5.09561088, -425.        ],
       [   6.58809867, -375.        ],
       [   5.00707005, -325.        ],
       [   5.84208227, -275.        ],
       [   4.97893122, -225.        ],
       [   3.74349973, -175.        ],
       [   4.50835795, -125.        ],
       [   2.89050684,  -75.        ],
       [   2.68265828,  -25.        ],
       [   4.05324237,   25.        ],
       [   3.16588101,   75.        ],
       [   3.44610747,  125.        ],
       [   3.13553061,  175.        ],
       [   3.03347991,  225.        ],
       [   2.53854072,  275.        ],
       [   2.65665718,  325.        ],
       [   1.74611679,  375.        ],
       [   3.78763209,  425.        ],
       [   2.8851597 ,  475.        ]])

### Plotting and saving figures for the binned bead localization plots

In [76]:
SAVE_DIR = Path('/mnt/sda1/SMLAT/figures/main/beads_crlb')

In [77]:
save_x = SAVE_DIR / Path('lprecision_x.pdf')
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.errorbar(bin_means_x[:, 1], bin_means_x[:, 0], yerr=bin_std_x[:, 0], fmt='bo', label='Bead data', capsize=3)
ax.plot(z, crlb[:, 0], 'r--', label = r'$\sqrt{CRLB_{x}}$ (nm) Spline model')
ax.legend(loc='upper right', fontsize=12)
ax.set_xlim([-500, 500])
ax.set_ylim([0, 50])
ax.set_xlabel('Z (nm)', fontsize=16)
ax.set_ylabel('Localization precision in X (nm)', fontsize=16)
ax.tick_params(labelsize=16)
fig.savefig(save_x, bbox_inches='tight', pad_inches=0, transparent=True, dpi=300)
#plt.close(fig)
plt.show()

In [78]:
save_x = SAVE_DIR / Path('lprecision_y.pdf')
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.errorbar(bin_means_y[:, 1], bin_means_y[:, 0], yerr=bin_std_y[:, 0], fmt='go', label='Bead data', capsize=3)
ax.plot(z, crlb[:, 1], 'r--', label = r'$\sqrt{CRLB_{y}}$ (nm) Spline model')
ax.legend(loc='upper right', fontsize=12)
ax.set_xlim([-500, 500])
ax.set_ylim([0, 50])
ax.set_xlabel('Z (nm)', fontsize=16)
ax.set_ylabel('Localization precision in Y (nm)', fontsize=16)
ax.tick_params(labelsize=16)

fig.savefig(save_x, bbox_inches='tight', pad_inches=0, transparent=True, dpi=300)
#plt.close(fig)
plt.show()

In [79]:
save_x = SAVE_DIR / Path('lprecision_z.pdf')
fig, ax = plt.subplots(nrows=1, ncols=1)
ax.errorbar(bin_means_z[:, 1], bin_means_z[:, 0], yerr=bin_std_z[:, 0], fmt='ko', label='Bead data', capsize=3)
ax.plot(z, crlb[:, 2], 'r--', label = r'$\sqrt{CRLB_{z}}$ (nm) Spline model')
ax.legend(loc='upper right',fontsize=12)
ax.set_xlim([-500, 500])

ax.set_xlabel('Z (nm)', fontsize=16)
ax.set_ylabel('Localization precision in Z (nm)', fontsize=16)
ax.tick_params(labelsize=16)

fig.savefig(save_x, bbox_inches='tight', pad_inches=0, transparent=True, dpi=300)
#plt.close(fig)
plt.show()

### Image stack saved from EXP-24-CB4719/fluorChannelRegistration3D/zStack_locError_103 Position 123 for the figure

In [80]:
from skimage.io import imread

In [57]:
stack_dir = Path('/mnt/sda1/SMLAT/figures/main/beads_crlb/stack/')

In [58]:
images_filenames = sorted(list(stack_dir.glob('*.png')))

In [59]:
images = [imread(filename) for filename in images_filenames]

In [67]:
for i, image in enumerate(images, 0):
    save_path = Path('/mnt/sda1/SMLAT/figures/main/beads_crlb/stack_cmapped/') / Path(images_filenames[i].name)
    fig, ax = plt.subplots(nrows=1, ncols=1)
    ax.imshow(image, cmap='gray')
    ax.set_axis_off()
    fig.savefig(save_path, bbox_inches='tight', pad_inches=0, transparent=True, dpi=300)
    plt.close(fig)
    plt.show()